This notebook scripts demonstrates the training process of the model introduced at [arXiv:2001.03622](https://arxiv.org/abs/2001.03622).
The model consists of two parts, which are quantum embedding and variational classifier.

In [1]:
import numpy as np
from pyquil_circuits import PauliFeatureMap, VariationalCircuit
from QVC import PyquilVariationalClassifier
from utils import load_data, bf

We employ the quantum embedding part with Pauli feature map, represented at [arXiv:1804.11326](https://arxiv.org/abs/1804.11326).
You can find our implementation of the pauli feature map [here](https://github.com/QuNovaComputing/Hackathon2021/blob/qunovacomputing/Qunova%20Computing/pyquil_circuits.py#L101).
We have the dataset with the dimension of 3, we put 3 as the number of qubits.

In [2]:
qfm = PauliFeatureMap(num_qubits=3, rep=2)
print(qfm.circuit)

DECLARE data REAL[3]
H 0
RZ(data[0]) 0
H 1
RZ(data[1]) 1
H 2
RZ(data[2]) 2
CNOT 0 1
RZ((pi - data[0])*(pi - data[1])) 1
CNOT 0 1
CNOT 1 2
RZ((pi - data[1])*(pi - data[2])) 2
CNOT 1 2
H 0
RZ(data[0]) 0
H 1
RZ(data[1]) 1
H 2
RZ(data[2]) 2
CNOT 0 1
RZ((pi - data[0])*(pi - data[1])) 1
CNOT 0 1
CNOT 1 2
RZ((pi - data[1])*(pi - data[2])) 2
CNOT 1 2
DECLARE ro BIT[3]
MEASURE 0 ro[0]
MEASURE 1 ro[1]
MEASURE 2 ro[2]



For the variational classifier, we used the circuit as below. The specific implementation can be found [here](https://github.com/QuNovaComputing/Hackathon2021/blob/qunovacomputing/Qunova%20Computing/pyquil_circuits.py#L130).

In [3]:
vc = VariationalCircuit(num_qubits=3, rep=2)
print(vc.circuit)

DECLARE param REAL[16]
RY(param[0]) 0
RY(param[1]) 1
RZ(param[2]) 0
RZ(param[3]) 1
CZ 0 1
RY(param[4]) 1
RY(param[5]) 2
RZ(param[6]) 1
RZ(param[7]) 2
CZ 1 2
RY(param[8]) 0
RY(param[9]) 1
RZ(param[10]) 0
RZ(param[11]) 1
CZ 0 1
RY(param[12]) 1
RY(param[13]) 2
RZ(param[14]) 1
RZ(param[15]) 2
CZ 1 2
DECLARE ro BIT[3]
MEASURE 0 ro[0]
MEASURE 1 ro[1]
MEASURE 2 ro[2]



We load the data with the number of principle comonents of 3. Also, we divide the data into the training data(80%) and test data(20%).
Specifying the random seed, we can repeat the experiment with the same condition.

In [4]:
seed = 30
train_data, train_labels, test_data, test_labels = load_data(test_size=0.2, num_PCs=3, seed=seed)
print(f"# of training data = {len(train_data)}")
print(f"# of test data     = {len(test_data)}")

# of training data = 81
# of test data     = 21


Combining two parts, we construct the variational classifier.

In [5]:
qvc = PyquilVariationalClassifier(qfm, vc, bool_ftn=bf, use_bias=False)

Start training. We use the optimizer with [SPSA](https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.SPSA.html),
with initial values with [1.0, ...].

To monitor the process, you can use `tensorboard`.
`tensorboard --logdir=./runs/zzzpfm_c12v3_zzzpfm_c12v3_pyquil`

In [6]:
print("Start training")

point, value, nfev = qvc.train(train_data, (-1) ** train_labels, 'zzzpfm_c12v3_pyquil',
                               test_data=test_data, test_label=(-1) ** test_labels,
                               spsa_maxiter=250)
print("Training Done")
print(f"optimal params       = {point}")
print(f"final training loss  = {value}")
print(f"function evaluations = {nfev}")

Start training
Training Done
optimal params       = [-0.779126    2.18269376  0.04466904  0.15724436 -0.8130034   0.6916049
  2.22174957  1.19136551  1.306887    0.42488094  0.93505012 -0.45540283
  0.74377079  0.35898721  0.92695085  1.26341594]
final training loss  = 0.8395061728395061
function evaluations = 750


Save the training result.

In [7]:
np.save('./npy_files/TrainData_zzpfmc12_pyquil.npy', train_data)
np.save('./npy_files/TestData_zzpfmc12_pyquil.npy', test_data)
np.save('./npy_files/TrainLabels_zzpfmc12_pyquil.npy', train_labels)
np.save('./npy_files/TestLabels_zzpfmc12_pyquil.npy', test_labels)
np.save('./npy_files/Optimal_param_zzpfmc12_pyquil.npy', point)

We have the training process as below. The x-axis corresponds to the iteration number, and the y-axis indicates the losses.

![loss_plot.jpg](./figures/loss_plot.jpg)